In [1]:
import torch
from datasets import IndexedDataset, WeightedDataset, load_data
from torch.utils.data import DataLoader, DistributedSampler

from utils import get_args
from architectures import load_architecture

from samplers import DistributedCustomSampler
from losses import trades_loss, apgd_loss
from tqdm.notebook import tqdm
from architectures import CustomModel, load_architecture, add_lora, set_lora_gradients #load_statedict

import torch.nn as nn
from torch.nn.parallel import DistributedDataParallel as DDP

args = get_args()

args.dataset = 'Flowers'
args.selection_method = 'random'
args.aug = 'aug'
args.loss_function = 'APGD'

args.iterations = 10
args.pruning_ratio = 0
args.delta = 1
args.batch_size = 24
args.init_lr = 0.001
args.freeze_epochs = 5
args.backbone = 'convnext_tiny' #deit_small_patch16_224.fb_in1k
args.ft_type = 'full_fine_tuning'

train_dataset, val_dataset, test_dataset, N, train_transform, transform = load_data(args) 
# print(N)

train_dataset = WeightedDataset(args, train_dataset, train_transform, N, prune_ratio = args.pruning_ratio,  )

# # train_sampler = DistributedCustomSampler(args, train_dataset, num_replicas=2, rank=0, drop_last=True)
train_sampler = DistributedSampler(train_dataset, num_replicas=2, rank=0, shuffle=True, drop_last=True)

trainloader = DataLoader(train_dataset, batch_size=3, sampler = train_sampler, )

# Example Usage

# rank = 0
model = load_architecture(args, N= 100, rank=0 )
# model = CustomModel(args, model)

# model.set_fine_tuning_strategy()
# model.to(rank)
# model = DDP(model, device_ids=[rank])

./data


In [2]:
from distributed_experiment1 import BaseExperiment

world_size = torch.cuda.device_count()
# set_seeds(args.seed)

experiment = BaseExperiment(args, world_size)

experiment.training(rank=0)

set up the distributed setup, 0


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.


torch 2.2.2+cu121
cuda 12.1
cudnn 8902
 world size 1, rank 0
set up the master adress and port
init process group ok


COMET INFO: Experiment is live on comet.com https://www.comet.com/maxheuillet/robust-training20/7a67287d12a24a43a51db5ade66a89e4



initialize dataset 0
initialize sampler 0
initialize dataoader 0
start the loop
effective batch size 1024 per_gpu_batch_size 8 accumulation steps 128
start batches
comput loss
 loss computed
average loss
divide loss
loss modification performed
scale loss and backward


/home/mheuillet/Desktop/robust_training/.venv/lib/python3.8/site-packages/torch/autograd/__init__.py:266: UserWarning: Grad strides do not match bucket view strides. This may indicate grad was not created according to the gradient layout contract, or that the param's strides changed since DDP was constructed.  This is not an error, but may impair performance.
grad.sizes() = [768, 1, 7, 7], strides() = [49, 1, 7, 1]
bucket_view.sizes() = [768, 1, 7, 7], strides() = [49, 49, 7, 1] (Triggered internally at ../torch/csrc/distributed/c10d/reducer.cpp:322.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


scale loss and backward done
gradient norm: 1.1932249135537547 rank: 0 loss tensor(0.0628, device='cuda:0', grad_fn=<DivBackward0>)
comput loss
 loss computed
average loss
divide loss
loss modification performed
scale loss and backward
scale loss and backward done
gradient norm: 1.8531544966043745 rank: 0 loss tensor(0.0693, device='cuda:0', grad_fn=<DivBackward0>)
comput loss
 loss computed
average loss
divide loss
loss modification performed
scale loss and backward
scale loss and backward done
gradient norm: 2.6430655788701953 rank: 0 loss tensor(0.0681, device='cuda:0', grad_fn=<DivBackward0>)
comput loss
 loss computed
average loss
divide loss
loss modification performed
scale loss and backward
scale loss and backward done
gradient norm: 3.2362334252894214 rank: 0 loss tensor(0.0651, device='cuda:0', grad_fn=<DivBackward0>)
comput loss
 loss computed
average loss
divide loss
loss modification performed
scale loss and backward
scale loss and backward done
gradient norm: 3.9770439142

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : APGD_0.001_sched_Flowers_convnext_tiny_random_0_random_10_0
COMET INFO:     url                   : https://www.comet.com/maxheuillet/robust-training20/7a67287d12a24a43a51db5ade66a89e4
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     global_step [1280]   : (1, 1280)
COMET INFO:     gradient_norm [1280] : (0.06997922581440406, 87.44580765550384)
COMET INFO:     loss_value [1280]    : (0.7767572999000549, 9.421733856201172)
COMET INFO:     lr_schedule [1280]   : (2.4471741852423235e-05, 0.001)
COMET INFO:     val_clean_accuracy   : 0.00980392156862745
COMET INFO:     val_loss [2]         : (8.185188802083333, 17.933517156862745)
COMET INFO:   

Model saved by rank 0
start the loop 3


In [2]:
from losses import get_loss, get_eval_loss
import numpy as np
from losses import apgd_loss

rank = 'cuda'

optimizer = torch.optim.SGD( model.parameters(),lr=args.init_lr, weight_decay=args.weight_decay, momentum=args.momentum, nesterov=True, )

from autoattack import autopgd_base
from torchvision.transforms import v2

cutmix = v2.CutMix(num_classes=N)
mixup = v2.MixUp(num_classes=N)
cutmix_or_mixup = v2.RandomChoice([cutmix, mixup])

for iteration in range(args.iterations):

    model.eval()
    train_sampler.set_epoch(iteration)

    # apgd = autopgd_base.APGDAttack(model, n_restarts=5, n_iter=args.perturb_steps, verbose=False,
    #             eps=args.epsilon, norm=args.distance, eot_iter=1, rho=.75, seed=args.seed,
    #             device='cuda', logger=None).perturb(data, target)
    
    for batch_id, batch in tqdm(enumerate( trainloader ) ):

        optimizer.zero_grad()

        data, target, idxs = batch

        print(data.shape, target.shape, target)

        data, target_one_hot = cutmix_or_mixup(data, target)

        print(target.shape, data.shape)

        # print(data,target, idxs)

        data, target = data.to(rank), target.to(rank) 

        print(target,)

        break

    break

        # data, target_one_hot = cutmix_or_mixup(data, target)

        # adv_data = apgd
        # print('data attacked')

        # loss_values, logits = get_loss(args, model, data, target, optimizer)

        # loss = loss_values.mean()
        # #loss = #train_dataset.compute_loss(idxs, loss_values)
        # print(loss)

        # loss.backward()
        # optimizer.step()

        # break
    
    # model.update_fine_tuning_strategy(iteration)

0it [00:00, ?it/s]

torch.Size([3, 3, 224, 224]) torch.Size([3]) tensor([44,  9, 38])
torch.Size([3]) torch.Size([3, 3, 224, 224])
tensor([44,  9, 38], device='cuda:0')


In [ ]:
from sklearn.model_selection import train_test_split

# Extract labels from the dataset
labels = [ label for _, label in dataset ]

# Split the dataset into train+val and test, keeping stratification
train_val_indices, test_indices = train_test_split( range(len(labels)), test_size=0.2, stratify=labels, random_state=42 )

# Extract labels for the train+val set for further stratification
train_val_labels = [labels[i] for i in train_val_indices]

# Split the train+val set into train and validation, keeping stratification
train_indices, val_indices = train_test_split( train_val_indices, test_size=0.15, stratify=train_val_labels, random_state=42 )  # 0.25 * 0.8 = 0.2 of the dataset

# Create subsets for train, validation, and test

N = 10

train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)
test_dataset = torch.utils.data.Subset(dataset, test_indices)

100%|██████████| 94280567/94280567 [00:15<00:00, 6066822.16it/s]


Extracting ./data/eurosat/EuroSAT.zip to ./data/eurosat


Dataset EuroSAT
    Number of datapoints: 27000
    Root location: ./data

In [ ]:
import timm

pretrained_models = timm.list_models(pretrained=True)

for m in pretrained_models:
    print(m)


bat_resnext26ts.ch_in1k
beit_base_patch16_224.in22k_ft_in22k
beit_base_patch16_224.in22k_ft_in22k_in1k
beit_base_patch16_384.in22k_ft_in22k_in1k
beit_large_patch16_224.in22k_ft_in22k
beit_large_patch16_224.in22k_ft_in22k_in1k
beit_large_patch16_384.in22k_ft_in22k_in1k
beit_large_patch16_512.in22k_ft_in22k_in1k
beitv2_base_patch16_224.in1k_ft_in1k
beitv2_base_patch16_224.in1k_ft_in22k
beitv2_base_patch16_224.in1k_ft_in22k_in1k
beitv2_large_patch16_224.in1k_ft_in1k
beitv2_large_patch16_224.in1k_ft_in22k
beitv2_large_patch16_224.in1k_ft_in22k_in1k
botnet26t_256.c1_in1k
caformer_b36.sail_in1k
caformer_b36.sail_in1k_384
caformer_b36.sail_in22k
caformer_b36.sail_in22k_ft_in1k
caformer_b36.sail_in22k_ft_in1k_384
caformer_m36.sail_in1k
caformer_m36.sail_in1k_384
caformer_m36.sail_in22k
caformer_m36.sail_in22k_ft_in1k
caformer_m36.sail_in22k_ft_in1k_384
caformer_s18.sail_in1k
caformer_s18.sail_in1k_384
caformer_s18.sail_in22k
caformer_s18.sail_in22k_ft_in1k
caformer_s18.sail_in22k_ft_in1k_384
c

NameError: name 'model' is not defined

In [5]:
effective_batch_size = 1024
world_size = 4
per_gpu_batch_size = 320

print( effective_batch_size // (world_size * per_gpu_batch_size), (world_size * per_gpu_batch_size), effective_batch_size) 

0 1280 1024


In [6]:
from architectures.resnet_imagenet import ResNet_imagenet, Bottleneck_imagenet
model = ResNet_imagenet(Bottleneck_imagenet, [3, 4, 6, 3], )
num_features = model.fc.in_features
print(num_features)
model.fc = nn.Linear(num_features, 10)

2048
